<a href="https://colab.research.google.com/github/samob917/en553.602.FA23/blob/main/Imputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


The purpose of this file is to impute values for budget based on a round robin regression on all features of the dataset.
---



In [3]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import csv
import numpy as np
from math import exp
import ast

In [6]:
# Initialize csv file paths
csv_file_path= 'final_train_dataset_with_categorical.csv'
# Create DataFrames from both CSV files
df = pd.read_csv(csv_file_path)

In [29]:
# Assuming 'df' is your existing dataframe
df['Merged Budget'] = df['Merged Budget'].replace(100000, np.nan)

selected_columns = [
    'IMDB Budget', 'TMDB Budget', 'TMDB Revenue', 'Runtime', 'IMDB Domestic Revenue',
    'international_revenue', 'worldwide_revenue', 'Merged Budget', 'Merged Revenue',
    'Release Year', 'Unweighted Star Score', 'Simple Weight Star Score',
    'Log Weight Star Score', 'Exponential Weight Star Score',
    'Movie Contribution to Director and Production Scores', 'Total Director Score',
    'Avg Director Score', 'Total Production Company Score',
    'Avg Production Company Score', 'Genre Cluster', 'Domestic Distributor ID',
    'Domestic Distributor Score', 'G', 'NC-17', 'NR', 'PG', 'PG-13', 'R',
    'Adj Merged Budget', 'Adj Merged Revenue', 'Has Star Score',
    'Has Director Score', 'Has Production Company Score',
    'Has Domestic Distributor Score', 'Unweighted Star Score_normalized',
    'Simple Weight Star Score_normalized', 'Log Weight Star Score_normalized',
    'Exponential Weight Star Score_normalized', 'Total Director Score_normalized',
    'Avg Director Score_normalized', 'Total Production Company Score_normalized',
    'Avg Production Company Score_normalized', 'Domestic Distributor Score_normalized'
]
fit_df = df[selected_columns]
# Remove rows where 'Merged Budget' is NaN
fit_df = fit_df.dropna(subset=['Merged Budget'])

# Create a new dataframe with selected columns, skipping the first row
new_df = df[selected_columns]
# Replace values equal to 100000 in the 'Merged Budget' column with empty values

# Display the updated dataframe
new_df.head()  # Use .head() to display the first few rows for verification


,IMDB Budget,TMDB Budget,TMDB Revenue,Runtime,IMDB Domestic Revenue,international_revenue,worldwide_revenue,Merged Budget,Merged Revenue,Release Year,...,Has Domestic Distributor Score,Unweighted Star Score_normalized,Simple Weight Star Score_normalized,Log Weight Star Score_normalized,Exponential Weight Star Score_normalized,Total Director Score_normalized,Avg Director Score_normalized,Total Production Company Score_normalized,Avg Production Company Score_normalized,Domestic Distributor Score_normalized
1,35000000,56000000,34737199,109,19701164,15036035,34737199.0,35000000.0,19701164,2013,...,1,1.625458,1.909947,2.052669,2.116033,1.517545,1.537691,0.698288,0.449979,1.123415
4,0,7000000,257019,89,257019,257019,NaN,7000000.0,257019,2013,...,0,-1.750039,-1.391208,-1.057694,-0.869858,1.010863,-1.911306,0.509202,0.694398,-0.914777
6,22000000,22000000,15400000,116,11330849,4330705,15661554.0,22000000.0,11330849,2013,...,1,0.965420,1.069109,1.075233,1.152906,-0.138206,-0.226494,1.371420,1.139955,0.601439
7,10000000,10000000,68500000,92,28873374,39694467,68567841.0,10000000.0,28873374,2013,...,1,0.529707,0.179257,-0.903549,0.072643,-0.438587,-0.902728,0.612487,0.811563,0.480839
9,15000000,15000000,146497771,100,71628180,74800000,146428180.0,15000000.0,71628180,2013,...,1,0.338834,0.492078,0.546700,0.587964,-0.989890,0.208833,1.107927,1.314570,1.095556


In [32]:
imp = IterativeImputer(max_iter=100000, random_state=1)
imp.fit(fit_df)
IterativeImputer(random_state=1)
imp_df = imp.transform(new_df)
imp_df = pd.DataFrame(imp_df, columns=new_df.columns)
imp_df.head()

,IMDB Budget,TMDB Budget,TMDB Revenue,Runtime,IMDB Domestic Revenue,international_revenue,worldwide_revenue,Merged Budget,Merged Revenue,Release Year,...,Has Domestic Distributor Score,Unweighted Star Score_normalized,Simple Weight Star Score_normalized,Log Weight Star Score_normalized,Exponential Weight Star Score_normalized,Total Director Score_normalized,Avg Director Score_normalized,Total Production Company Score_normalized,Avg Production Company Score_normalized,Domestic Distributor Score_normalized
0,0.0,0.0,0.0,83.0,2241.0,2241.0,1.280029e+05,-6.174014e+04,2241.0,2013.0,...,0.0,-1.197882,-0.834900,-1.914023,-1.492851,0.020794,0.140460,-1.351097,-0.771759,-0.075203
1,35000000.0,56000000.0,34737199.0,109.0,19701164.0,15036035.0,3.473720e+07,3.500000e+07,19701164.0,2013.0,...,1.0,1.625458,1.909947,2.052669,2.116033,1.517545,1.537691,0.698288,0.449979,1.123415
2,0.0,0.0,0.0,90.0,134049.0,134049.0,3.947623e+05,-6.476363e+04,134049.0,2013.0,...,0.0,-1.223715,-0.523574,-0.754037,-0.503238,0.323499,0.158286,-0.666598,-0.458559,-0.887904
3,0.0,0.0,1553826.0,106.0,1561962.0,1561962.0,3.265744e+06,-1.006970e+05,1561962.0,2013.0,...,0.0,-0.308416,-0.624991,-0.869441,-0.118806,-0.179807,-1.146126,-0.893549,-1.395688,-0.991378
4,0.0,7000000.0,257019.0,89.0,257019.0,257019.0,6.305445e+05,7.000000e+06,257019.0,2013.0,...,0.0,-1.750039,-1.391208,-1.057694,-0.869858,1.010863,-1.911306,0.509202,0.694398,-0.914777


In [34]:
fin_df = pd.read_csv(csv_file_path)

# Selecting only the 'Merged Budget' column from imp_df
imp_df_merged_budget = imp_df[['Merged Budget']]

# Replace 'Merged Budget' column in 'fin_df' with the values from 'imp_df_merged_budget'
fin_df['Merged Budget'] = imp_df_merged_budget['Merged Budget']

# Save the modified DataFrame 'fin_df' as a CSV file
fin_df.to_csv('imputed_budget.csv', index=False)